In [1]:
import os
import pprint
import tempfile
import datetime
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [2]:
train_filename = "../data/samples/train_transactions.tfrecord"
train = tf.data.TFRecordDataset(train_filename)

test_filename = "../data/samples/test_transactions.tfrecord"
test = tf.data.TFRecordDataset(test_filename)

validation_filename = "../data/samples/validation_transactions.tfrecord"
validation = tf.data.TFRecordDataset(validation_filename)

feature_description = {
    'context_item_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),    
    'context_item_quantity': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),
    'context_item_price': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),    
    'context_item_department_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'label_item_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
}

def _parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, feature_description)
def _map_function(x):
  return {
    "context_item_id": tf.strings.as_string(x["context_item_id"]),
    "context_item_price": x["context_item_price"],
    "context_item_department_id": x["context_item_department_id"],
    "label_item_id": tf.strings.as_string(x["label_item_id"])}
    
train_ds = train.map(_parse_function).map(_map_function)
test_ds = test.map(_parse_function).map(_map_function)
cached_train = train_ds.shuffle(10_000).batch(12800).cache()
cached_test = test_ds.batch(2560).cache()
cached_validation = validation.map(_parse_function).map(_map_function).batch(1280).cache()

In [3]:
items_filename = "../data/samples/items.tfrecord"
items_tf = tf.data.TFRecordDataset(items_filename)
item_feature_description = {
    'item_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
    'item_fullprice': tf.io.FixedLenFeature([1], tf.float32, default_value=0.0 )
}

def item_parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, item_feature_description)

items_ds = items_tf.map(item_parse_function).map(lambda x: {
    "item_id": tf.strings.as_string(x["item_id"]),
    "item_fullprice": x["item_fullprice"]
})
item_ids = items_ds.map(lambda x: x["item_id"]).batch(1_000)
unique_item_ids = np.unique(np.concatenate(list(item_ids)))
item_prices = np.concatenate(list(items_ds.map(lambda x: x["item_fullprice"]).batch(1000)))

dept_filename = "../data/samples/departments.tfrecord"
departments_tf = tf.data.TFRecordDataset(dept_filename)
dept_feature_description = {
  'dept_id' : tf.io.FixedLenFeature([1], tf.int64, default_value=0)
}
def dept_parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, dept_feature_description)
department_ids = np.concatenate(list(departments_tf.map(dept_parse_function).map(lambda x: x['dept_id'])))

In [4]:
class ItemEmbeddingModel(tf.keras.Model):
  embedding_dimension = 32
  def __init__(self):
    super().__init__()

    self.item_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(vocabulary=unique_item_ids, mask_token=None, name='item_id_string_lookup'),
      tf.keras.layers.Embedding(len(unique_item_ids) + 1, 64, name='item_id_embedding'),
      tf.keras.layers.GRU(48, name='item_id_rnn', return_sequences=True),
      tf.keras.layers.GRU(self.embedding_dimension, name='item_id_rnn_out')
    ], name='item_embedding')

    self.price_normalization = tf.keras.layers.Normalization(axis=-1)

    self.price_embedding = tf.keras.Sequential([
      self.price_normalization,
      tf.keras.layers.Embedding(len(item_prices)+1,output_dim=64, mask_zero=True),
      tf.keras.layers.GRU(48, name='item_price_rnn', return_sequences=True),
      tf.keras.layers.GRU(self.embedding_dimension, name='item_price_rnn_out')
    ], name='price_embedding')    

    self.price_normalization.adapt(item_prices)

    self.dept_categorical = tf.keras.layers.IntegerLookup(max_tokens=len(department_ids)+1, vocabulary=department_ids)

    self.dept_embedding = tf.keras.Sequential([
      self.dept_categorical,
      tf.keras.layers.Embedding(len(department_ids)+1, output_dim=64, mask_zero=True),
      tf.keras.layers.GRU(48, name='item_dept_rnn', return_sequences=True),
      tf.keras.layers.GRU(self.embedding_dimension, name='item_dept_rnn_out')
    ])

  def call(self, features):
    return tf.concat([
        self.item_embedding(features["context_item_id"]),
        self.price_embedding(features["context_item_price"]),
        self.dept_embedding(features['context_item_department_id'])
    ], axis=1)

class DeepLayerModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(self, layer_sizes, embedding_model):
    """Model for encoding user queries.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = embedding_model

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [5]:
class RetrievalModel(tfrs.Model):
    def __init__(self, deep_layers):
        self.embedding_dim = 32
        super().__init__()
        self._query_model = DeepLayerModel(deep_layers, ItemEmbeddingModel())
        self._candidate_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_item_ids, mask_token=None, name='candidate_itemid_lookup'),
            tf.keras.layers.Embedding(len(unique_item_ids) + 1, self.embedding_dim, name='candidate_embedding_lookup'),
            tf.keras.layers.Dense(self.embedding_dim)
            ], name='candidate_model')
        metrics = tfrs.metrics.FactorizedTopK(candidates=items_tf.batch(128).map(self._candidate_model))
        self._task = tfrs.tasks.Retrieval(metrics=metrics)

    def compute_loss(self, features, training=False):
        item_history = {
            "context_item_id": features["context_item_id"],
            "context_item_price": features["context_item_price"],
            "context_item_department_id" : features["context_item_department_id"]}   
        next_item_label = features["label_item_id"]

        query_embedding = self._query_model(item_history)       
        candidate_embedding = self._candidate_model(next_item_label)

        return self._task(query_embedding, candidate_embedding, compute_metrics=not training)

In [6]:
log_dir = '../logs/fit/stacked_gru_model'
deep_layers = [32]
epochs = 12
learning_rate = 0.9
model = RetrievalModel(deep_layers=deep_layers)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=learning_rate))
model.fit(cached_train, epochs=epochs, validation_data=cached_validation, callbacks=[tensorboard_callback])
model.fit(cached_train, epochs=epochs)
evaluation = model.evaluate(cached_test, return_dict=True)
print(evaluation)

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model._query_model)
# recommends movies out of the entire movies dataset.
items = items_ds.map(lambda x: x["item_id"])
index.index_from_dataset(
  tf.data.Dataset.zip((items.batch(100), items.batch(100).map(model._candidate_model)))
)
feature0 = ['139716','35287','142953','132041','','','','','','']
feature1 = [17.99, 14.99, 4.99, 14.50, 0.,0.,0.,0.,0.,0.]
feature2 = [736,670,753,735,0,0,0,0,0,0]
features_dataset = { 
  'context_item_id' : tf.constant(feature0,shape=(1,10)), 
  'context_item_price' : tf.constant(feature1,shape=(1,10)),
  'context_item_department_id' : tf.constant(feature2, shape=(1,10))
}

scores, predicted = index(features_dataset)
print(scores)
print(predicted)

tf.Tensor(
[[33.03748  31.948565 31.939894 31.650888 31.61806  31.369473 31.27339
  31.186274 31.047937 31.03676 ]], shape=(1, 10), dtype=float32)
tf.Tensor(
[[[b'148660']
  [b'153464']
  [b'133217']
  [b'96375']
  [b'129016']
  [b'142292']
  [b'69045']
  [b'127485']
  [b'135720']
  [b'105703']]], shape=(1, 10, 1), dtype=string)


In [ ]:
path = '../data/stacked_rnn_model'
tf.saved_model.save(index, path)

KeyboardInterrupt: 